![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
qb = QuantBook()
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
#import statsmodels
#import statsmodels.api as sm
#from statsmodels.tsa.stattools import coint, adfuller
#from statsmodels import regression,stats
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
#from xml.etree import ElementTree as ET
#from IPython.core.debugger import set_trace
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [2]:
# Using 90-day lookback history to calculate VaR
# The start date is 9/24/2019 because we have 62 trading days betweetn 2/1/2020 and 5/1/2020.
# 90 + 62 = 152
# 152 trading days between 9/24/2019 and 5/1/2020
start_time = datetime(2019, 9, 24) # start datetime for history call
end_time = datetime(2020, 5, 1) # end datetime for history call

In [3]:
gs = qb.AddEquity("GS", Resolution.Daily)
ms = qb.AddEquity("MS", Resolution.Daily)
aapl = qb.AddEquity("AAPL", Resolution.Daily)
msft = qb.AddEquity("MSFT", Resolution.Daily)
symbol = ["GS", "MS", "AAPL", "MSFT"]
history = qb.History(qb.Securities.Keys, start_time, end_time, Resolution.Daily)

In [5]:
df = history['close'].unstack(level=0)

In [6]:
df.shape # 62 + 90 = 152

In [7]:
df

In [26]:
df1 = history.loc["GS"]
df2 = history.loc["MS"]
df3 = history.loc["AAPL"]
df4 = history.loc["MSFT"]
df1
a = pd.DataFrame()
a['X'] = df1["close"]
a['Y'] = df2["close"]
a

In [23]:
d1 = pd.DataFrame()
d1["GS"] = df1['close']
d1[]


In [8]:
df1

In [9]:
df1['log'] = df1['close'].apply(lambda x: math.log(x))
df2['log'] = df2['close'].apply(lambda x: math.log(x))
df3['log'] = df3['close'].apply(lambda x: math.log(x))
df4['log'] = df4['close'].apply(lambda x: math.log(x))

In [10]:
df1['log_diff'] = df1['log'] - df2['log']
df3['log_diff'] = df3['log'] - df4['log']
df1

In [32]:
Z = pd.DataFrame()
Z = df1['log'] - df2['log']
Z
Z[0]

In [23]:
np.sqrt(Z['spread'])

In [62]:
df1['log_diff']

In [63]:
slope, intercept, r_value, p_value, std_err = stats.linregress(df2['log'],df1['log'])

plt.figure(figsize=(16, 7))
p= sns.regplot(x = df2['log'], y = df1['log'], color = 'navy',
            line_kws={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)})
print(" Hedge Ratio = Slope of the Linear Regression Equation: Y = " + str('{:.2f}'.format(slope))+ " X + " + str('{:.2f}'.format(intercept)))

In [65]:
df1['W'] = df1['log'] - slope * df2['log']
df1['W'].plot(figsize = [16,7], title ='LnP - hedge ratio * LnQ', legend = True)

In [67]:
sigma = np.std(df1['W'])
mu = df1['W'].mean()
df1['μ+σ'] = mu + sigma
df1['μ-σ'] = mu - sigma
df1['W'].plot(figsize = [16,7], title ='LnP - hedge ratio * LnQ', legend = True)
df1['μ+σ'].plot(figsize = [16,7], linestyle = ':', legend = True)
df1['μ-σ'].plot(figsize = [16,7], linestyle = ':', legend = True)

In [134]:
cash = 100000
margin = 0.3
df1.shape[0]
PnL = pd.DataFrame(index = df1.index, columns = ["PnL", "pos1", "pos2"])

In [135]:
PnL.iloc[[1]]

In [202]:
flag = 0 # flag = 1 means we already enter, flag = 0 means we don't have position
for i in range(df1.shape[0]):
    if flag == 0:
        # portfolio weights for ticker1 and ticker2
        wt1 = 1/(1 + slope)
        wt2 = slope/(1 + slope)
        pos1 = round(wt1 * cash/(df1.close[i] * margin))
        pos2 = round(wt2 * cash/(df2.close[i] * margin))
        # When W > μ+σ, long Q and short P
        if df1['W'][i] > df1['μ+σ'][i]:
            PnL.iloc[i]["PnL"] = df2.close[i] * pos2 - df1.close[i] * pos1
            PnL.iloc[i]["pos1"] = -pos1
            PnL.iloc[i]["pos2"] = pos2
            flag = 1 # indicate we already enter
        # When W < μ-σ, long P and short Q
        elif df1['W'][i] < df1['μ-σ'][i]:
            PnL.iloc[i]["PnL"] = df1.close[i] * pos1 - df2.close[i] * pos2
            PnL.iloc[i]["pos1"] = pos1
            PnL.iloc[i]["pos2"] = -pos2
            flag = 1 # indicate we already enter
    
    elif flag == 1:
        # When W = μ, Liquidate all the positions
        if abs(df1['W'][i] - mu) < 0.001:
            PnL.iloc[i]["PnL"] = 0
            PnL.iloc[i]["pos1"] = 0
            PnL.iloc[i]["pos2"] = 0
            flag = 0 # indicate we already exit
        else: # We hold our positions
            PnL.iloc[i]["pos1"] = PnL.iloc[i-1]["pos1"]
            PnL.iloc[i]["pos2"] = PnL.iloc[i-1]["pos2"]
            if PnL.iloc[i-1]["pos2"] > 0:
                PnL.iloc[i]["PnL"] = df2.close[i] * PnL.iloc[i]["pos2"] - PnL.iloc[i]["pos1"] * PnL.iloc[i]["pos1"]
            else:
                PnL.iloc[i]["PnL"] = df1.close[i] * PnL.iloc[i]["pos1"] - df2.close[i] * PnL.iloc[i]["pos2"]
    

In [203]:
print(PnL)

In [204]:
PnL["PnL"].plot(figsize = [16,7], x = 'date', marker = "*")


In [205]:
port_n = PnL.PnL-PnL.PnL.shift(1)
port_n.hist(figsize = [16,7], bins =30)
#port_n is the change in portfolio values between n days.

In [206]:
port_n.plot(figsize = [16,7], x = 'date', marker = "*")

In [207]:
port_n

In [212]:
VaR = pd.DataFrame(index = df1.index, columns = ["VaR"])[89:]
VaR

In [218]:
alpha = 0.95 # confidence interval
for i in range(63):
    port = pd.DataFrame(data = port_n[i:89 + i])
    port['pct_rank'] = port.rank(pct = True)
    pct_n = port[port.pct_rank < 1-alpha] # Find the tail distribution
    var_n= pct_n.PnL.max() # the cut off point is the VaR
    avg_var_n = pct_n.PnL.mean() # average of the tail is tail var
    sum_t = (pct_n['pct_rank']).sum()
    VaR.iloc[i] = var_n
    print(pct_n)
    print(var_n)

In [221]:
VaR